# Ingress Services Lab

### Introduction

In this lesson, we'll use what we learned about ingress services to allow external access to our cluster.  Let's get started.

### Setting it Up

Now, if you have not yet installed the ingress-nginx service,  now is a good time.  Remember that the documentation for doing so.  

We first run `minikube addons list` which should show us the list of addons.

And then run `minikube addons enable ingress`.

Now this will install a number of resources in the kube-system namespace.  So we can see this by viewing the pods now available in the `kube-system` namespace.  Run the following to take a look:

`kubectl get pods -n kube-system`

> <img src="./show_ingress.png" width="60%">

And importantly, we can see that now if we visit the ip address of our cluster, we'll now see this point to nginx.  We can view the ip address of our cluster by visiting `minikube ip`.

> <img src="./nginx_404.png" width="60%">

Next, remember that we'll to setup our `/etc/hosts` file so that a request to a domain name of `apartments.com` goes to our cluster's ip address.

> `sudo vim /etc/hosts`

Then configure the file appropriately.  Now, if we visit `apartments.com` we should see the same nginx screen.  

> **Warning**: We may have to view the domain while in incognito mode, as otherwise our browser may rely on a cached request. 

### Adding to our ingress

Ok, now it's time to add an ingress service.  We'll do so such that we can make external requests to the frontend-service and the backend-service.  The frontend service should be requested through the root url -- `/`, and the backend service should be connected through a request to `apartments.com/api`.

> We'll provide the frontend service for you, which you simply need to add to a yaml file, and apply the changes.

```yaml
apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
    name: public-routing
spec:
    rules:
        - host: apartments.com
          http:
            paths:
                - path: /
                  pathType: Prefix
                  backend: 
                    service:
                      name: frontend-service
                      port: 
                        number: 8501
```

Then confirm that you can access the streamlit service by visiting the ip address of the cluster.

> It may take a number of minutes to boot up, check the `get pods` and `describe pods` command to check the progress. 

> <img src="./working-application.png" width="40%">

The frontend service is of type NodePort.  As we know, this makes the service publicly accessible.  Update the service so that it is only accessible from within the cluster, and check that our website is still working.

Ok, now let's add a route to the backend service.

We'll want any requests to `/api` to be transferred to the backend service.  We'll also want to change the backend service from type NodePort so that the service is not directly accessible.

Now we cannot get this working completely at this point, but we'll know that we're on the right track if we see the following:

> <img src="./proper_404.png" width="80%">

The above is the 404 for the flask application.  Now this isn't exactly what we hope to see.  We'd like to see the corresponding json.

We can get a sense of what's occurring by looking at the logs for our backend pods.  Run the following to view the logs, and you may see something like the following.

<img src="./problem_with_api.png" width="80%">

So notice above that the request is coming into our api pods as `/api/apartments`.  Instead, we would prefer the nginx ingress to get the `/api` request, and this request to be forwarded onto the api as just  `/apartments`.

It turns our there is some setup to accomplish that.

### Rewrite Targets

Now to get this working we'll use a rewrite target.  This means that our ingress service will get a request and rewrite the request path before passing it along to the corresponding service.  There is some fairly good documentation on how this works [here](https://kubernetes.github.io/ingress-nginx/examples/rewrite/).

So we can accomplish this by updating our nginx service with the following.  Change the first few fields of the `ingress.yaml` file to the following:

```yaml
apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
    name: public-routing
    annotations:
      nginx.ingress.kubernetes.io/rewrite-target: /$1
```

And update the path related to the api service so that instead of `path: /api` it now says:

```yaml
- path: /api/?(.*)
```

Ok, now let's explain what the above accomplishes starting the with path.

The path of `/api/?(.*)` is a regex.  And it says that it finds a match of `api/` followed by subsequent characters.  With the `(.*)`, the parentheses are a capture group -- and the `.*` with will match any character for one or more times.  So the `(.*)` will essentially capture any characters occurring after the `/api`.  We reference this as capture group 1.

Now in the config file above, we added the following:

```yaml
annotations:
      nginx.ingress.kubernetes.io/rewrite-target: /$1
```

This means to rewrite the request to the backend with whatever is captured in capture group 1, here, anything after the `/api`.  So a request to `/api/apartments` will now be passed along to the backend as `/apartments`.

Update the `ingress.yaml` file, and apply the changes.  Now if we make a request to /api/apartments, we can check the logs and see that it's forwarded onto our pod as `/apartments`.

> <img src="./apartments.png" width="60%">

And in the browser, we should now be able to go to `/api/apartments` and see the json.


> <img src="./working_json.png" width="100%">

### Summary

### Resources
[Debug Services](https://kubernetes.io/docs/tasks/debug-application-cluster/debug-service/)

[K8s Workflow](https://www.getambassador.io/resources/dev-workflow-intro/)

[Telepresence](https://www.telepresence.io/tutorials/kubernetes-rapid)

